In [2]:
import numpy as np
from tribarmes import TriangleBarMesh

In [16]:
fname = "stlsamples/frameguide.stl"
#fname = "stlsamples/onshapetestpart.stl"
tbm = TriangleBarMesh(fname)
vs = tbm.GetBarMeshTriangles()

# quickly make these as independent triangles
vv = np.array(vs).ravel()
X, Y, Z = vv[0::3], vv[1::3], vv[2::3]
tris = np.array(range(len(vs)*3))
tris.resize((len(vs), 3))


In [17]:
# define a cutter shape
cornerrad, flatrad, shaftheight = 5, 1, 38
px, py, pz = 20, -20, 0

tr = np.linspace(0, cornerrad+1, cornerrad*10+1)
theta = np.linspace(0, 2*np.pi, 40)
tr, theta = np.meshgrid(tr, theta)
def toolheight(tr):
    return cornerrad - np.sqrt(np.maximum(0, cornerrad**2 - tr**2)) + \
                       np.maximum(0, (tr-cornerrad)*(shaftheight))
tX = np.minimum(tr+flatrad, cornerrad+flatrad) * np.sin(theta)
tY = np.minimum(tr+flatrad, cornerrad+flatrad) * np.cos(theta)
tZ = toolheight(tr)

from matplotlib.tri import Triangulation
ttri = Triangulation(np.ravel(tr), np.ravel(theta))
ttris = ttri.triangles
    

In [14]:
import ipyvolume as ipv

def rendersurftool(px, py):
    k = ipv.view()
    print(k)
    ipv.figure()
    ipv.xyzlim(-40,40)
    mesh = ipv.plot_trisurf(X, Y, Z, triangles=tris, color='orange')
    ipv.plot_trisurf(tX.ravel()+px, tY.ravel()+py, tZ.ravel()+pz, triangles=ttris, color="red")
    ipv.view(*k)
    ipv.show()

rendersurftool(20, -20)
#ipv.save("/home/julian/repositories/transition-CAM/thing.html", offline=True)


(52.15934314565082, -27.55988551614986, 1.6641985619143262)


In [10]:
from ipywidgets import FloatSlider, interact, HBox, interactive_output

fsx = FloatSlider(value=20, min=-50, max=50, step=1, description="xpos", continuous_update=False)
fsy = FloatSlider(value=-20, min=-50, max=50, step=1, description="ypos", continuous_update=False)

out = interactive_output(rendersurftool, {"px":fsx, "py":fsy})
display(HBox([fsx, fsy]), out)


Output()

In [17]:
import ipyvolume as ipv
import numpy as np
x, y, z, u, v, w = np.random.random((6, 1000))*2-1
selected = np.random.randint(0, 1000, 100)
ipv.figure()
quiver = ipv.quiver(x, y, z, u, v, w, size=5, size_selected=8, selected=selected)

from ipywidgets import FloatSlider, ColorPicker, VBox, jslink
size = FloatSlider(min=0, max=30, step=0.1)
size_selected = FloatSlider(min=0, max=30, step=0.1)
color = ColorPicker()
color_selected = ColorPicker()
jslink((quiver, 'size'), (size, 'value'))
jslink((quiver, 'size_selected'), (size_selected, 'value'))
jslink((quiver, 'color'), (color, 'value'))
jslink((quiver, 'color_selected'), (color_selected, 'value'))
VBox([ipv.gcc(), size, size_selected, color, color_selected])